
# Advent of Code 2023

> The effort of using machines to mimic the human mind has always struck me as rather silly. I would rather use them to mimic something better.

-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
from urllib import request
from functools import reduce, cache
from itertools import product, accumulate, count, pairwise, repeat, cycle
from operator import or_, matmul, xor, add
from math import inf, prod, isqrt, lcm
from dataclasses import dataclass
from collections import Counter, defaultdict
import re


def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2023/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: Trebuchet?!](https://adventofcode.com/2023/day/1)

In [2]:
d = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

regexp = re.compile(f'(?=(\\d|{"|".join(d)}))')


def find_digits(s):
    return [int(i) for i in s if i.isdigit()]


def find_digit_names(s):
    def to_number(p):
        if p.isdigit():
            return int(p)
        else:
            return d[p]

    return [to_number(p) for p in regexp.findall(s)]


input_1 = aocin(1).split('\n')

A = sum(
    10 * x[0] + x[-1]
    for x in map(find_digits, input_1)
)
assert A == 55123

A = sum(
    10 * x[0] + x[-1]
    for x in map(find_digit_names, input_1)
)
assert A == 55260

## [Day 2: Cube Conundrum](https://adventofcode.com/2023/day/2)

In [3]:
@dataclass
class Game:
    gameid: int
    contents: list[Counter[str, int]]

        
def line_to_game(l):
    num, items = l.split(':')
        
    _, gameid = num.split()
        
    contents = [
        Counter({b: int(a) for a, b in [y.split() for y in x.split(',')]})
        for x in items.split(';')
    ]
        
    return Game(int(gameid), contents)


games = list(map(line_to_game, aocin(2).split('\n')))

A = sum(
    game.gameid 
    for game in games
    if all(
        d < Counter({'red': 12, 'green': 13, 'blue': 14}) 
        for d in game.contents
    )
)
assert A == 2476

A = sum(prod(reduce(or_, game.contents).values()) for game in games)
assert A == 54911

## [Day 3: Gear Ratios](https://adventofcode.com/2023/day/3)

In [4]:
class Grid:
    def __init__(self, g):
        self._grid = g
        self.columns = len(g[0])
        self.rows = len(g)

    def __getitem__(self, t):
        x, y = t
        return self._grid[y][x]
    
    def adjacent(self, x, y):
        return [
            self[i, j] 
            for i, j in {
                (x+1, y), (x-1, y), (x, y+1), (x, y-1),
                (x+1, y+1), (x-1, y+1), (x+1, y-1), (x-1, y-1)
            }
            if 0 <= i < self.columns and 0 <= j < self.rows
        ]
    
    @classmethod
    def from_text(cls, lines):
        return cls([list(x) for x in lines])
    

@dataclass
class PartNumber:
    value: int
    col_start: int
    col_end: int
    row: int


def find_part_numbers(grid, is_symbol):
    val, symbol_adjacent = 0, False
    for j in range(grid.rows):
        for i in range(grid.columns):
            c = grid[i, j]
            if c.isdigit():
                val = 10 * val + int(grid[i, j])
            if not c.isdigit():
                if val and symbol_adjacent:
                    yield PartNumber(
                        val, i - len(str(val)), i - 1, j
                    )
                val, symbol_adjacent = 0, False
            if val and any(is_symbol(s) for s in grid.adjacent(i, j)):
                symbol_adjacent = True
        if val and symbol_adjacent:
            yield PartNumber(
                val, grid.columns - len(str(val)), grid.columns - 1, j
            )
        val, symbol_adjacent = 0, False
        
        
def find_gear_ratios(grid):
    part_numbers = defaultdict(list)
    for p in find_part_numbers(grid, lambda c: c == '*'):
        part_numbers[p.row].append(p)
    
    for i, j in product(range(grid.columns), range(grid.rows)):
        if grid[i, j] != '*':
            continue
                
        adjacents = (
            [
                p for p in part_numbers[j] 
                if p.col_start == i+1 or p.col_end == i-1
            ] + [
                p for p in part_numbers[j-1] + part_numbers[j+1]
                if p.col_start <= i+1 and p.col_end >= i-1
            ]
        )
            
        if len(adjacents) == 2:
            yield adjacents
       

grid = Grid.from_text(aocin(3).split('\n'))
            
A = sum(
    p.value 
    for p in find_part_numbers(grid, lambda c: not (c.isdigit() or c == '.'))
)
assert A == 536576
    
A = sum(a.value * b.value for a, b in find_gear_ratios(grid))
assert A == 75741499

## [Day 4: Scratchcards](https://adventofcode.com/2023/day/4)

Part 1 is very simple. The only observation is that it's one of the many declinations of the count distinct problem: let the sets of winning numbers and of the player's numbers be $M$ and $N$ respectively, then $|M \cap N| = |M| + |N| - |M \cup N|$. This has a worst-case lower bound of $\Theta(n \log n)$ [(for proof see: Grigoriev 99)](https://www.semanticscholar.org/paper/Complexity-lower-bounds-for-randomized-computation-Grigoriev/7b0c914a951bc59bb0cee33f197e6418a3c2600b), which may be achieved in a variety of ways.

The code implements the expected linear time randomized solution of merging two hash-sets (technically, its worst-case complexity is $\Theta(n^2)$, but that happens with probability 0).

Part 2 is also very simple. Let $n$ be the number of cards, and let $m$ be the maximum number of numbers appearing in any given card.

Winning cards generate more cards with higher numbers, and the hypotheses of the problem ensure that only copies of cards in the initial range are generated. This is sufficient to easily prove termination, and it suggests a naive algorithm: 

- initialize an array $A[1..n]$ with $A[i] = 1$ for all $1 \le i \le n$, where $A[i]$ represents the copies of card $i$ we have, initially one.
- for all $1 \le j \le n$, compute how many winning numbers $w_i$ the $i$-th card has, and for all $j+1 \le k \le j+w$, let $A[k] \leftarrow A[k] + A[j]$
- the final answer is $\sum A[i]$.

If we let $f(m)$ be the complexity of the subroutine that computes how many numbers a given card has, it's easy to see that the above algorithm has complexity bounded by $\Theta(n(m + f(m)))$. But because $f(m) \in \Omega(m)$, this is already optimal.

### Day 4 - Bonus Round

Imagine that instead of having to compute the matching numbers on the cards, we are instead given an oracle for that, in other words, $f(m) \in \Theta(1)$. Can we do better than $\Theta(mn)$? We sure can, thanks to the prefix trick!

Let us initialize an array $A[1..n+1]$ with $A[i] = 0$ for all $2 \le i \le n$, $A[1] = 1$ and $A[n+1] = -1$.

The idea is that the number of copies of card $i$ we have is represented by $\sum_{j=0}^i A[j]$. If we wish to increase by $h$ the amount of copies of cards $r+1, r+2, \dots, r+w$, it is sufficient to let $A[r+1] \leftarrow A[r+1] + h$ and $A[r+w+1] \leftarrow A[r+w+1] - h$.

Therefore, for each $1 \le j \le n$, we let $w_j$ be the winning numbers of the $j$-th card, and $c_j$ be the number of copies of that card. We may simply set $A[j+1] \leftarrow A[j+1] + c_j$ and $A[j+w_j+1] \leftarrow A[r+w_j+1] - c_j$.

At each loop iteration, $c_j$ may be computed as $c_{j-1} + A[j]$ by definition.

The final answer will be:

$$
      \sum_{i=1}^{n} \sum_{j=1}^{i} A[i]
    = \sum_{i=1}^{n+1} \left( \sum_{j=1}^{n+1} A[j] - \sum_{j=i+1}^{n+1} A[j] \right)
    = (n+1) \left( \sum_{i=1}^{n+1} A[i] \right) -  \left( \sum_{i=1}^{n+1} \sum_{j=i}^{n+1} A[i] \right)
    = \sum_{i=1}^{n+1} (n-i+1) A[i]
$$

The initialization and final computation are linear-time operations, and each loop iteration is constant time, which yields an algorithm of complexity $\Theta(nf(m))$.

This could have been such a beautiful problem :-(


In [5]:
@dataclass
class Scratchcard:
    cardid: int
    winning: set[int]
    my: set[int]
    
    def winning_numbers(self):
        return len(self.winning & self.my)

    def score(self):
        s = self.winning_numbers()
        return s and 2 ** (s-1)
    
    @classmethod
    def from_line(cls, l):
        num, items = l.split(':')
        
        _, cardid = num.split()
        
        winning, my = (
            {int(y) for y in x.split()}
            for x in items.split('|')
        ) 
        
        return cls(int(cardid), winning, my)


def recursive_scratchcards(cards):
    cards_array = [c.winning_numbers() for c in cards]
    prefix_array = [1] + [0] * (len(cards_array) - 1) + [-1]
    for n, (w, c) in enumerate(zip(cards_array, accumulate(prefix_array))):
        prefix_array[n+1] += c
        prefix_array[n+w+1] -= c
    return sum(n * c for n, c in enumerate(reversed(prefix_array)))
 

scratchcards = list(map(Scratchcard.from_line, aocin(4).split('\n')))
        
A = sum(c.score() for c in scratchcards)
assert A == 23941

A = recursive_scratchcards(scratchcards)
assert A == 5571760

## [Day 5: If You Give A Seed A Fertilizer](https://adventofcode.com/2023/day/5)

The mapping represented by the problem is a series of piecewise linear maps $f_1, f_2, \dots f_n$. The crucial observation is this: because a linear map sends compacts into compacts, then a piecewise linear map sends a finite union of compacts into a finite union of compacts.

The composition of two piecewise linear maps may be computed entirely symbolically, as can the image of a finite union of compacts through it. 

It is easy to see that the combination of these two sub-problems solves the original problem: one may compute the union of the starting sets (seeds) $U = U_1 \cup U_2 \dots \cup U_n$, the composition $F = f_n \circ f_{n-1} \circ \dots \circ f_1$ and the final answer $\inf F(U)$. Alternatively, we may directly compute $f_n(f_{n-1}( \dots f_1(x) \dots ))$.

### Lemma: Computing the intersections of two sorted sets of pairwise disjoint intervals

Let $\iota_i = [\alpha_i, \beta_i)$ be a $m$ pairwise disjoint intervals and $\kappa_i = [\phi_i, \psi_i)$ be $n$ pairwise disjoint intervals. Further, assume that $\alpha_1 \le \beta_1 \le \alpha_2 \le \beta_2 \le \dots \le \alpha_n \le \beta_n$ and likewise $\phi_1 \le \psi_1 \le \phi_2 \le \psi_2 \le \dots \le \phi_n \le \psi_n$.

Their intersections may be computed in time bounded by $\Theta(n + m)$, and they are at most $n + m$.

This may be done by performing a merge join:

- Let $a \leftarrow 1, b \leftarrow 1$.
- While $a \le m \wedge b \le n$, compute $I = \iota_a \cap \kappa_b$ and report it if $I \neq \emptyset$, then increment $a$ if $\beta_a \le \psi_b$, increment $b$ otherwise.

This works because the while loop preserves the following invariant: all unreported intersections lie at the right of $\min \alpha_a, \beta_b$.

To see that there are at most $n + m$ intersections it is sufficient to observe that intersections may only be created at boundary points.

### Step 1: Computing the composition of two piecewise linear maps

Let $f$ and $g$ be two piecewise linear maps, that is, functions in the form:

$$
f(x) = \left\{
    \begin{array}{ll}
        x + c_1 & \text{if } x \lt \alpha_1 \\
        x + c_2 & \text{if } \alpha_1 \le x \lt \alpha_2 \\
        \dots \\
        x + c_{m-1} & \text{if } \alpha_{m-1} \le x \lt \alpha_{m} \\
        x + c_m & \text{if } \alpha_m \le x \\
    \end{array}
\right.
$$

$$
g(x) = \left\{
    \begin{array}{ll}
        x + d_1 & \text{if } x \lt \beta_1 \\
        x + d_2 & \text{if } \beta_1 \le x \lt \beta_2 \\
        \dots \\
        x + d_{n-1} & \text{if } \beta_{n-1} \le x \lt \beta_{n-1} \\
        x + d_n & \text{if } \beta_n \le x \\
    \end{array}
\right.
$$

for arbitrary constants $\{c_i\}, \{d_i\}$, $m$ pairwise disjoint intervals $A_i = [\alpha_i, \alpha_{i+1})$, and $n$ pairwise disjoint intervals $B_i = [\beta_i, \beta_{i+1})$. For the sake of convenience, without loss of generality, we take all intervals to be closed on the left and open on the right.

We wish to compute $g \circ f$.

First, we sort the $B$. This takes time $\Theta(n \log n)$. Then, we compute and sort the images through $\Lambda_i = f(A_i)$ of each of the intervals on which $f$ is defined. This takes time $\Theta(m \log m)$. The intersection of these two sorted sets may be computed using the lemma.

But if $\Sigma_{ij} = \Lambda_i \cap A_j \neq \emptyset$ for some $i, j$, that means on the preimage $f^{-1}(\Sigma_{ij})$ it must hold $(g \circ f)(x) = x + c_i + d_j$.

Repeated over all intersections, this does in fact determine $g \circ f$ as a piecewise linear function.

### Step 2: Computing the image of a finite union of intervals through a piecewise linear map

We can use the same idea as before. Let the intervals be $U_1, U_2, \dots U_n$ with $U$ being their union, and let the piecewise linear map be $f$ with the same notation as the above paragraph.

We sort the $U_i$ and the $A_i$ and find their intersections.

If $\Sigma_{ij} = A_i \cap U_j \neq \emptyset$ for some $i, j$, then on $\Sigma_{ij}$ it holds $f(x) = x + c_i$.

The union of all $f(\Sigma_{ij})$ is exactly $f(U)$. Those intervals may be overlapping, but it's very easy to find their "clean" union: sort the intervals by their initial point, look at them in order, and replace each pair of overlapping intervals with their union by resizing the endpoints.

### Complexity

Both for the direct computation option (use Step 2 to compute $f_n(f_{n-1}( \dots f_1(x) \dots ))$ directly) and for the composition option (use Step 1 to compute $F = f_n \circ f_{n-1} \circ \dots \circ f_1$, then use Step 2 to compute $F(U)$), the complexity depends on the number of intersections.

If the procedure from the lemma produces $s$ intersections, the resulting output union of intervals (if we are using the first option) or the map composition (if we are using the second option) will have $s$ intervals. Because we know that intersecting sets of intervals of sizes $m$ and $n$ produces at most $n+m$ intersections, we conclude that both variants of the algorithm have complexity $\Theta(h \log h)$ where $h$ is the sum of the number of intervals of all the original piecewise linear maps.

### Day 5 - Bonus Round
Because all of this only uses properties of linear maps, it can be generalized further to "actual" linear maps, where each of the cases can be in the form $f(x) = ax + b$ with relatively little effort. And, best of all, it works on _real numbers_ with the only additional effort of bookkeeping on the interval boundaries.

In [6]:
@dataclass
class IntRange:
    lo: int | float
    hi: int | float

    __bool__ = lambda s: s.lo < s.hi
    __lt__ = lambda s, o: (s.lo, s.hi) < (o.lo, o.hi)
    __and__ = lambda s, o: IntRange(max(s.lo, o.lo), min(s.hi, o.hi))


@dataclass
class LinearMap:
    delta: int
  
    __call__ = lambda s, x: IntRange(x.lo + s.delta, x.hi + s.delta)
    __invert__ = lambda s: LinearMap(-s.delta)
    __matmul__ = lambda s, o: LinearMap(s.delta + o.delta)
    I = lambda: LinearMap(0)


class IntRangeUnion:
    def __init__(self, pieces):
        self.union = []
        for p in sorted(pieces):
            if not self.union:
                self.union.append(p)
            elif p.lo <= self.union[-1].hi:
                self.union[-1].hi = p.hi
            else:
                self.union.append(p)
    
    @staticmethod 
    def from_list(seq):
        return IntRangeUnion([
            IntRange(seq[l], seq[l] + seq[l+1])
            for l in range(0, len(seq), 2)
        ])

    def inf(self):
        return self.union[0].lo


class PiecewiseLinearMap:
    I = lambda: PiecewiseLinearMap([(IntRange(-inf, +inf), LinearMap.I())])
    
    def __init__(self, pieces):
        self.pieces = sorted(pieces)
    
    @classmethod 
    def from_list(cls, m):
        l = sorted([
            (IntRange(source, source+length), LinearMap(dest - source))
            for dest, source, length in m
        ])
        
        return cls(l + [
            (IntRange(-inf, l[0][0].lo), LinearMap.I()),
            (IntRange(l[-1][0].hi, inf), LinearMap.I())
        ])
        
    def __matmul__(self, other):
        def merge():
            l = iter(sorted(self.pieces, key=lambda u: u[1](u[0])))
            r = iter(other.pieces)
            (a, f), (b, g) = next(l, (None, None)), next(r, (None, None))
            while a and b:
                if s := (f(a) & b):
                    yield (~f)(s), f @ g
                if f(a).hi < b.hi:
                    a, f = next(l, (None, None))
                else:
                    b, g = next(r, (None, None))
                
        return PiecewiseLinearMap(merge()) 
   
    def __call__(self, x):
        def merge():
            l, r = iter(self.pieces), iter(x.union)
            (a, f), b = next(l, (None, None)), next(r, None)
            while a and b:
                if s := a & b:
                    yield f(s)
                if a.hi < b.hi:
                    a, f = next(l, (None, None))
                else:
                    b = next(r, None)
          
        return IntRangeUnion(merge())



input_5 = aocin(5).split('\n\n')
seeds = [int(y) for y in input_5[0].split(':')[1].split()]
maps = [
    [
        tuple(int(t) for t in y.split())
        for y in x.split('\n')[1:]
    ]
    for x in input_5[1:]
]

parsed_maps = list(map(PiecewiseLinearMap.from_list, maps))
piecewise_map = reduce(matmul, parsed_maps, PiecewiseLinearMap.I())
map_composition = reduce(lambda u, v: lambda x: v(u(x)), parsed_maps, lambda u: u)

part_1_input = IntRangeUnion(IntRange(a, a+1) for a in seeds)
A = piecewise_map(part_1_input).inf()
B = map_composition(part_1_input).inf()
assert A == B == 510109797

part_2_input = IntRangeUnion.from_list(seeds)
A = piecewise_map(part_2_input).inf()
B = map_composition(part_2_input).inf()
assert A == B == 9622622

## [Day 6: Wait For It](https://adventofcode.com/2023/day/6)

Solvable entirely with pen and paper!

If the available time is $t$, the initial press of the button has length $x$ and the record is $d$, the integer values of $x$ that beat the record are those that satisfy

$$
x(t-x) > d
$$

The quadratic equation has solutions:

$$
x_{1,2} = \frac{t \pm \sqrt{t^2 - 4d}}{2}
$$

The interval $(x_1, x_2)$ contains exactly $\left \lfloor x_1 \right \rfloor - \left \lfloor x_2 \right \rfloor$ integers... unless either $x_1$ or $x_2$ are integers, in which case it contains one less.

The interesting part is that _all_ of the problem, as the following implementation shows, may be solved without ever using floating point arithmetic, using the following two facts:

- $\left \lfloor \sqrt{x} \right \rfloor = \text{isqrt}(x)$
- $\left \lceil \sqrt{x} \right \rceil = 1 + \text{isqrt}(x-1)$

In [7]:
def integer_solutions(t, d):
    fld = isqrt(t**2 - 4*d)
    cld = isqrt(t**2 - 4*d - 1) + 1 
    x1 = (t + fld) // 2
    x2 = (t - cld) // 2
   
    if fld != cld:
        return x1 - x2
    
    return x1 - x2 + 2 + ((t + fld) % 2) + ((t - cld) % 2)


time, distance = (x.split()[1:] for x in aocin(6).split('\n'))

A = prod(map(integer_solutions, map(int, time), map(int, distance)))
assert A == 2756160

A = integer_solutions(int(''.join(time)), int(''.join(distance)))
assert A == 34788142

## [Day 7: Camel Cards](https://adventofcode.com/2023/day/7)

In [8]:
def combination(cards, wild=None):
    c = Counter(cards)
    n_wilds = c.pop(wild, 0)
    s = sorted(c.values(), reverse=True) or [0]
    s[0] += n_wilds
    return s


class Hand:
    def __init__(self, cards, bid, wild=None):
        def card_value(s):
            if s.isdigit(): return int(s)
            if s == wild: return 0
            return 10 + 'TJQKA'.index(s)
            
        self.cards = [card_value(x) for x in cards]
        self.bid = int(bid)
        self.comb = combination(cards, wild)

    __lt__ = lambda s, o: (s.comb, s.cards) < (o.comb, o.cards)


input_7 = [x.split() for x in aocin(7).split('\n')]

hands = (Hand(cards, bid) for cards, bid in input_7)
A = sum(r * h.bid for r, h in enumerate(sorted(hands), 1))
assert A == 249390788

hands_j_wild = (Hand(cards, bid, wild='J') for cards, bid in input_7)
A = sum(r * h.bid for r, h in enumerate(sorted(hands_j_wild), 1))
assert A == 248750248

## [Day 8: Haunted Wasteland](https://adventofcode.com/2023/day/8)

A detailed analysis of this problem is available in its own notebook [here](https://nbviewer.org/github/edoannunziata/jardin/blob/master/misc/Aoc23Day8BonusRound.ipynb).

In [9]:
class StateAutomaton:
    def __init__(self, trans):
        self.trans = defaultdict(list)
        self.all_states = set()
        for pre, post, label in trans:
            self.trans[pre].append((post, label))
            self.all_states |= {pre, post}
            
    def transition(self, pre, label):
        for post, tlabel in self.trans[pre]:
            if label == tlabel:
                return post
            
    @classmethod
    def from_text(cls, ls):
        def gen_trans(ls):
            for l in ls:
                a, b, c = l.translate(
                    str.maketrans('(),=', '    ')
                ).split()
                yield a, b, 'L'
                yield a, c, 'R'
        return cls(gen_trans(ls))


@cache
def fast_ptr(automaton, state, labels):
    return reduce(automaton.transition, labels, state)


@cache
def cycle_length(automaton, state, labels):
    visited = {}
    for n in count():
        if state in visited:
            return visited[state], n - visited[state]
        visited[state] = n
        state = fast_ptr(automaton, state, labels) 


@cache
def accepted_points(automaton, state, labels, cond):
    def _trans_automaton(state):
        for n, l in enumerate(labels, 1):
            state = automaton.transition(state, l)
            if cond(state):
                yield n, state
    return list(_trans_automaton(state))


@dataclass
class AcceptedPoints:
    anticycle_accepted: list[int]
    anticycle_length: int
    cycle_accepted: list[int]
    cycle_length: int
        
    def gen(self):
        yield from iter(self.anticycle_accepted)
        for k in count():
            yield from (
                k * self.cycle_length + self.anticycle_length + x
                for x in self.cycle_accepted
            )


def accepted_points_repeating(automaton, state, labels, cond):
    anticycle, cycle = cycle_length(automaton, state, labels)
    
    anticycle_accepted = []
    for i in range(anticycle):
        anticycle_accepted += [
            i * len(labels) + n for n, _ in
            accepted_points(automaton, state, labels, cond)
        ]
        state = fast_ptr(automaton, state, labels)
    
    cycle_accepted = []
    for j in range(cycle):
        cycle_accepted += [
            j * len(labels) + n for n, _ in
            accepted_points(automaton, state, labels, cond)
        ]
        state = fast_ptr(automaton, state, labels)
        
    return AcceptedPoints(
        anticycle_accepted,
        anticycle * len(labels),
        cycle_accepted,
        cycle * len(labels)
    )


moves, _, *trans = aocin(8).split('\n')
S = StateAutomaton.from_text(trans)

A = next(accepted_points_repeating(S, 'AAA', moves, lambda u: u == 'ZZZ').gen())
assert A == 16579

cond = lambda u: u.endswith('Z')
starting_states = {a.split()[0] for a in trans if a.split()[0].endswith('A')}
accepted = {
    state: accepted_points_repeating(S, state, moves, cond)
    for state in starting_states
}
A = lcm(*(v.cycle_length for v in accepted.values()))
assert A == 12927600769609

This does not work in general! The input is in fact a special case where the following happens:

In [10]:
accepted

{'LJA': AcceptedPoints(anticycle_accepted=[], anticycle_length=281, cycle_accepted=[21918], cycle_length=22199),
 'AAA': AcceptedPoints(anticycle_accepted=[], anticycle_length=281, cycle_accepted=[16298], cycle_length=16579),
 'JXA': AcceptedPoints(anticycle_accepted=[], anticycle_length=281, cycle_accepted=[16860], cycle_length=17141),
 'NFA': AcceptedPoints(anticycle_accepted=[], anticycle_length=281, cycle_accepted=[11802], cycle_length=12083),
 'KTA': AcceptedPoints(anticycle_accepted=[], anticycle_length=281, cycle_accepted=[14612], cycle_length=14893),
 'PLA': AcceptedPoints(anticycle_accepted=[], anticycle_length=281, cycle_accepted=[19670], cycle_length=19951)}

## [Day 9: Mirage Maintenance](https://adventofcode.com/2023/day/9)

In [11]:
def finite_differences_next(l):
    s = [l]
    while any(s[-1]):
        s.append([b-a for a, b in pairwise(s[-1])])
    
    return sum(t[-1] for t in s)


def finite_differences_prev(l):
    s = [l]
    while any(s[-1]):
        s.append([b-a for a, b in pairwise(s[-1])])
        
    return reduce(lambda a, b: b-a, (t[0] for t in reversed(s)))


input_9 = [list(map(int, x.split())) for x in aocin(9).split('\n')]

A = sum(map(finite_differences_next, input_9))
assert A == 1819125966

A = sum(map(finite_differences_prev, input_9))
assert A == 1140

## [Day 10: Pipe Maze](https://adventofcode.com/2023/day/10)

Cute!

Annoying parsing aside, part 1 is trivial. To solve part 2, we compute the intersection number with the curve to determine if the center of each integer square is inside or outside the curve.

Alternative solution for the memes: use the Shoelace theorem to compute the area of the polygon, and Pick's theorem to compute how many integer points it contains.

In [12]:
@dataclass
class V:
    x: int
    y: int
    
    __add__ = lambda s, o: V(s.x+o.x, s.y+o.y)
    __sub__ = lambda s, o: V(s.x-o.x, s.y-o.y)
    __hash__ = lambda s: hash((s.x, s.y))


class PipeMaze:
    PIPES = {
        '-': {V(1, 0), V(-1, 0)},
        '|': {V(0, 1), V(0, -1)},
        'F': {V(1, 0), V(0, 1)},
        '7': {V(-1, 0), V(0, 1)},
        'J': {V(-1, 0), V(0, -1)},
        'L': {V(1, 0), V(0, -1)}
    }
    
    def __init__(self, g):
        self._grid = g
        self.columns = len(g[0])
        self.rows = len(g)
        
        for i, j in product(range(self.columns), range(self.rows)):
            if self[V(i, j)] != 'S':
                continue
                
            self.S = V(i, j)
                
            compat = {
                x for x in (V(0, 1), V(0, -1), V(1, 0), V(-1, 0))
                if self.is_compatible(self.S, self.S+x)
            }
        
            for pipe, cs in PipeMaze.PIPES.items():
                if compat == cs:
                    self._grid[self.S.y][self.S.x] = pipe
        
    def __getitem__(self, t):
        if 0 <= t.x < self.columns and 0 <= t.y < self.rows:
            return self._grid[t.y][t.x]

    def is_compatible(self, pre, post):
        return pre-post in PipeMaze.PIPES[self[post]]
    
    @classmethod
    def from_text(cls, lines):
        return cls([list(x) for x in lines])

            
def find_main_loop(g):
    this, prev = g.S, None
    
    while not prev or this != g.S:
        new = {
            this + x for x in PipeMaze.PIPES[g[this]]
            if this + x != prev
        }.pop()
        
        yield new
        this, prev = new, this
                
    
def interior_points(grid, loop):
    clean_grid = PipeMaze([
        [
            grid[V(i, j)] if V(i, j) in loop else '.'
            for i in range(grid.columns)
        ]
        for j in range(grid.rows)
    ])

    for j in range(clean_grid.rows):
        parity = False
        for i in range(clean_grid.columns):
            match clean_grid[V(i, j)], parity:
                case 'L' | 'J' | '|', _:
                    parity = not parity
                case '.', True:
                    yield i, j
    
    
def interior_points_pick(loop):            
    s = sum(
        loop[i].x * loop[(i+1) % len(loop)].y
        - loop[i].y * loop[(i+1) % len(loop)].x
        for i in range(len(loop))
    )
    
    return (abs(s) - len(loop)) // 2 + 1


grid = PipeMaze.from_text(aocin(10).split('\n'))
loop = list(find_main_loop(grid))

A = len(loop) // 2
assert A == 6864

A = sum(1 for _ in interior_points(grid, set(loop)))
B = interior_points_pick(loop)
assert A == B == 349

## [Day 11: Cosmic Expansion](https://adventofcode.com/2023/day/11)

Because we are working with the $L_1$ metric (taxicab distance), we may work on each dimension separately. In one dimension, if the galaxies have coordinates $a_1, a_2, \dots a_n$ with $a_1 < a_2 < \dots < a_n$, and there are $c_1$ galaxies at $a_1$, $c_2$ galaxies at $a_2$ and so on, the sum of all pairwise distances is simply:

$$
\sum_{i=1}^{n} \sum_{j=i}^{n} c_i c_j (a_j - a_i)
$$

Let

$$
\begin{array}{ll}
    W(a, b) &= \sum_{j=a}^b c_j \\
    T(i) &= \sum_{j=i}^n c_j (a_j - a_i)
\end{array}
$$

It holds:
$$ 
\begin{array}{ll}
    T(1) &= \sum_{j=1}^n c_j (a_j - a_1) \\
    T(h) &= T(h-1) - W(h, n) (a_h - a_{h-1}) 
\end{array}
$$

which suggests a $\Theta(n + m \log m)$ algorithm, where $n$ is the number of galaxies, and $m$ is the number of distinct coordinates.

In [13]:
def embiggen(g, k):
    def _embiggen_map(l):
        prev_src, prev_dest = -1, -1
        for i in l:
            n = prev_dest + k*(i-prev_src-1) + 1
            yield i, n
            prev_src, prev_dest = i, n

    return Counter({
        nk: g[ok] for ok, nk in _embiggen_map(sorted(g))
    })


def pairwise_distances_flat(g):
    w_after = sum(g.values())
    t_last, a_last = sum(k * v for k, v in g.items()), 0

    acc = 0
    for a in sorted(g):
        t_new = t_last - w_after * (a - a_last)
        acc += t_new * g[a]
        w_after -= g[a]
        t_last, a_last = t_new, a

    return acc


def pairwise_distances(g, enbiggen_by):
    return sum(
        pairwise_distances_flat(embiggen(p, enbiggen_by))
        for p in (Counter(x for x, _ in g), Counter(y for _, y in g))
    )


galaxies = {
    (i, j)
    for j, l in enumerate(aocin(11).split('\n'))
    for i, c in enumerate(l)
    if c == '#'
}

A = pairwise_distances(galaxies, enbiggen_by=2)
assert A == 9214785

A = pairwise_distances(galaxies, enbiggen_by=1_000_000)
assert A == 613686987427

## [Day 12: Hot Springs](https://adventofcode.com/2023/day/12)

Not difficult once you see it's greedy.

In [14]:
@dataclass
class Spring:
    cont: str
    spec: list

    @classmethod
    def from_text(cls, s):
        cont, spec = s.split()
        return cls(cont, list(map(int, spec.split(','))))

    def unfold(self, n):
        return Spring('?'.join(repeat(self.cont, n)), self.spec*n)


def num_sol(spring):
    d = Counter([(0, 0)])
    for c in spring.cont + '.':
        nd = Counter()
        for (stage, v), count in d.items():
            if c in '.?':
                if stage < len(spring.spec) and spring.spec[stage] == v:
                    nd[(1+stage, 0)] += count
                elif v == 0:
                    nd[(stage, v)] += count
            if c in '#?' and stage < len(spring.spec) and spring.spec[stage] > v:
                nd[(stage, 1+v)] += count
        d = nd

    return d[(len(spring.spec), 0)]


input_12 = aocin(12).split('\n')

A = sum(map(num_sol, map(Spring.from_text, input_12)))
assert A == 7169

A = sum(map(num_sol, (x.unfold(5) for x in map(Spring.from_text, input_12))))
assert A == 1738259948652

## [Day 13: Point of Incidence](https://adventofcode.com/2023/day/13)

In [15]:
class RockGrid:
    def __init__(self, grid):
        self.rows = grid
        self.columns = [''.join(x) for x in zip(*grid)]

    @classmethod
    def from_text(cls, lines):
        return cls([x for x in lines.split('\n')])


def to_number(s):
    return sum(2**i for i, c in enumerate(s) if c == '#') 


def mirrors_at(l):
    for n, h in enumerate(accumulate(map(to_number, l), xor)):
        if not h and l[:(n+1)//2] == l[(n+1)//2:n+1][::-1]:
            return (n + 1) // 2


def mirrors_at_err(l):
    ns = list(map(to_number, l))
    is_pow_two = lambda n: n and not (n & (n-1))

    def check_sol(n):
        prev = 0
        for a, b in zip(ns[:(n+1)//2], ns[(n+1)//2:n+1][::-1]):
            if a == b or not prev:
                prev = a ^ b or prev
            else:
                return False
        return is_pow_two(prev)
    
    for n, h in enumerate(accumulate(ns, xor)):
        if is_pow_two(h) and check_sol(n):
            return (n + 1) // 2


def score_mirror(g, f):
    if s := f(g.rows):
        return 100 * s
    if s := f(g.rows[::-1]):
        return 100 * (len(g.rows) - s)
    if s := f(g.columns):
        return s
    if s := f(g.columns[::-1]):
        return len(g.columns) - s


input_13 = aocin(13).split('\n\n')

A = sum(score_mirror(RockGrid.from_text(x), mirrors_at) for x in input_13)
assert A == 30802

A = sum(score_mirror(RockGrid.from_text(x), mirrors_at_err) for x in input_13)
assert A == 37876

## [Day 14: Parabolic Reflector Dish](https://adventofcode.com/2023/day/14)

In [16]:
class TiltingGrid:
    def __init__(self, grid, transpose=False):
        self.rows = grid
        self.columns = [''.join(x) for x in zip(*grid)]
        
        if transpose:
            self.rows, self.columns = self.columns, self.rows

    def tilt(self, dir):
        def counting_sort(s, order):
            c = Counter(s)
            return ''.join(x * c[x] for x in order)

        def tilt_line(col, order):
            pieces = col.split('#')
            return '#'.join(counting_sort(p, order) for p in pieces)
       
        if dir == 'n':
            return TiltingGrid([tilt_line(l, 'O.') for l in self.columns], True)
        if dir == 's':
            return TiltingGrid([tilt_line(l, '.O') for l in self.columns], True)
        if dir == 'w':
            return TiltingGrid([tilt_line(l, 'O.') for l in self.rows])
        if dir == 'e':
            return TiltingGrid([tilt_line(l, '.O') for l in self.rows])
    
    def score(self):
        return sum(
            len(self.rows) - j 
            for c in self.columns 
            for j, x in enumerate(c) 
            if x == 'O'
        )
    
    __hash__ = lambda s: hash(tuple(s.rows))


def tilt_many(grid, order, cycles):
    seen, grid_after = {}, {}
    for n, dir in enumerate(cycle(order), 1):
        grid = grid.tilt(dir)
        if (hash(grid), dir) in seen:
            anticycle_len = seen[(hash(grid), dir)]
            cycle_len = n - anticycle_len
            remaining = cycles * len(order) - n
            return grid_after[anticycle_len + remaining % cycle_len]
        seen[(hash(grid), dir)] = n
        grid_after[n] = grid


input_14 = aocin(14).split('\n')

A = TiltingGrid(input_14).tilt('n').score()
assert A == 105249

A = tilt_many(TiltingGrid(input_14), 'nwse', 10 ** 9).score()
assert A == 88680

## [Day 15: Lens Library](https://adventofcode.com/2023/day/15)

In [17]:
def str_hash(s):
    return reduce(lambda a, c: (17 * (a + ord(c))) % 256, s, 0)


def run_hashmap(instr):
    m = defaultdict(list)
    for i in instr:
        label, focal = i.replace('-', '=').split('=')
        lh = str_hash(label)
        ls = [i for i, (l, _) in enumerate(m[lh]) if label == l]
        match [*focal], ls:
            case [], [_ as idx]:
                m[lh].pop(idx)
            case [_], [_ as idx]:
                m[lh][idx] = (label, focal)
            case [_], []:
                m[lh].append((label, focal))
    return m


input_15 = aocin(15).split(',')

A = sum(map(str_hash, input_15))
assert A == 507666

A = sum(
    (1+k) * i * int(f)
    for k, v in run_hashmap(input_15).items()
    for i, (_, f) in enumerate(v, 1)
)
assert A == 233537

## [Day 16: The Floor Will Be Lava](https://adventofcode.com/2023/day/16)

In [18]:
class MirrorGrid:
    def __init__(self, g):
        self._grid = g
        self.columns = len(g[0])
        self.rows = len(g)
        
    __getitem__ = lambda s, t: s._grid[t[1]][t[0]]
    __contains__ = lambda s, t: 0 <= t[0] < s.columns and 0 <= t[1] < s.rows


def visit(g, start):
    seen = {start}
    S = [start]

    def try_append(x, y, dx, dy):
        if (x, y) in g and (x, y, dx, dy) not in seen:
            S.append((x, y, dx, dy))
            seen.add((x, y, dx, dy))
            
    while S:
        x, y, dx, dy = S.pop()
        match g[x, y], dx, dy:
            case '-', 0, _:
                try_append(x+1, y, 1, 0)
                try_append(x-1, y, -1, 0)
            case '/', _, _:
                try_append(x-dy, y-dx, -dy, -dx)
            case '\\', _, _:
                try_append(x+dy, y+dx, dy, dx)
            case '|', _, 0:
                try_append(x, y+1, 0, 1)
                try_append(x, y-1, 0, -1)
            case _:
                try_append(x+dx, y+dy, dx, dy)

    return {(x, y) for x, y, _, _ in seen}


def border_points(g):
    yield from ((i, 0, 0, 1) for i in range(g.columns))
    yield from ((i, g.rows-1, 0, 1) for i in range(g.columns))
    yield from ((0, i, 1, 0) for i in range(g.rows))
    yield from ((g.columns-1, i, -1, 0) for i in range(g.rows))


g = MirrorGrid(aocin(16).split('\n'))

A = len(visit(g, (0, 0, 1, 0)))
assert A == 7392

A = max(len(visit(g, s)) for s in border_points(g))
assert A == 7665

## [Day 17: Clumsy Crucible](https://adventofcode.com/2023/day/17)

In [19]:
class CrucibleGrid:
    def __init__(self, g):
        self._grid = [[int(c) for c in row] for row in g]
        self.columns = len(g[0])
        self.rows = len(g)
        
    __getitem__ = lambda s, t: s._grid[t[1]][t[0]]
    __contains__ = lambda s, t: 0 <= t[0] < s.columns and 0 <= t[1] < s.rows


class PriorityQueue:
    def __init__(self):
        self._H = []
        self._map = {}
        self._len = 0

    __bool__ = lambda s: s._len > 0
    __contains__ = lambda s, v: v in s._map
    _parent = lambda s, n: (n+1) // 2 - 1 if n else None

    def _children(self, n):
        return (
            2*n+1 if self._len >= 2*n+2 else None,
            2*n+2 if self._len > 2*n+2 else None
        )

    def _swap(self, a, b):
        self._map[self._H[a][1]], self._map[self._H[b][1]] = b, a
        self._H[a], self._H[b] = self._H[b], self._H[a]

    def _heap_decrease(self, n, r):
        self._H[n] = r
        p = self._parent(n)

        while p is not None and self._H[n] < self._H[p]:
            self._swap(n, p)
            n, p = p, self._parent(p)

    def insert(self, prio, val):
        self._H.append(None)
        self._map[val] = self._len
        self._heap_decrease(self._len, (prio, val))
        self._len += 1

    def extract(self):
        top_prio, top_val = self._H[0]
        self._swap(0, self._len - 1)
        del self._H[-1]
        del self._map[top_val]
        self._len -= 1
        c, (l, r) = 0, self._children(0)

        while l or r:
            cprio, _ = self._H[c]
            lprio, _ = self._H[l] if l else (inf, inf)
            rprio, _ = self._H[r] if r else (inf, inf)

            if cprio <= lprio and cprio <= rprio:
                break
            elif lprio < rprio:
                self._swap(c, l)
                c = l
            else:
                self._swap(c, r)
                c = r

            l, r = self._children(c)

        return top_prio, top_val

    def decrease_prio(self, newprio, val):
        idx = self._map[val]
        prio, _ = self._H[idx]
        self._heap_decrease(idx, (newprio, val))


def successors(g, p, dir, min, max):
    def section(p, d, dir):
        x, y = p
        dx, dy = d
        acc = 0
        for d in range(1, max+1):
            x, y = x+dx, y+dy
            if (x, y) not in g:
                break
            acc += g[x, y]
            if min <= d:
                yield (x, y), dir, acc

    if dir != 'H':
        yield from section(p, (1, 0), 'H')
        yield from section(p, (-1, 0), 'H')
    if dir != 'V':
        yield from section(p, (0, 1), 'V')
        yield from section(p, (0, -1), 'V')


def dijkstra(g, start, target, min, max):
    M = {(start, 'X'): 0}
    Q = PriorityQueue()
    Q.insert(0, (start, 'X'))

    while Q:
        v, (p, h) = Q.extract()
        if p == target:
            return v
                          
        for n, hn, vn in successors(g, p, h, min, max):                
            if v + vn >= M.get((n, hn), inf):
                continue

            M[(n, hn)] = v + vn

            if (n, hn) in Q:
                Q.decrease_prio(v + vn, (n, hn))
            else:
                Q.insert(v + vn, (n, hn))


grid = CrucibleGrid(aocin(17).split('\n'))

A = dijkstra(grid, (0, 0), (grid.columns-1, grid.rows-1), 1, 3)
assert A == 1044

A = dijkstra(grid, (0, 0), (grid.columns-1, grid.rows-1), 4, 10)
assert A == 1227

## [Day 18: Lavaduct Lagoon](https://adventofcode.com/2023/day/18)

Shoelace formula + Pick's theorem again. Straightforward implementation.

In [20]:
def vertices_one(l):
    def offset(instr):
        match instr.split():
            case 'R', n, _: return int(n), 0
            case 'L', n, _: return -int(n), 0
            case 'D', n, _: return 0, int(n)
            case 'U', n, _: return 0, -int(n)
    
    return list(accumulate(map(offset, l), lambda *t: tuple(map(add, *t)), initial=(0, 0)))


def vertices_two(l):
    def offset(instr):
        match divmod(int(instr.split()[2].strip('(#)'), 16), 16):
            case n, 0: return n, 0
            case n, 1: return 0, n
            case n, 2: return -n, 0
            case n, 3: return 0, -n
        
    return list(accumulate(map(offset, l), lambda *t: tuple(map(add, *t)), initial=(0, 0)))


def total_area(vs):
    A = sum(x1 * y2 - y1 * x2 for (x1, y1), (x2, y2) in pairwise(vs))
    B = sum(abs(x1 - x2) + abs(y1 - y2) for (x1, y1), (x2, y2) in pairwise(vs))
    return (A + B) // 2 + 1


input_18 = aocin(18).split('\n')

A = total_area(vertices_one(input_18))
assert A == 36807

A = total_area(vertices_two(input_18))
assert A == 48797603984357

## [Day 19: Aplenty](https://adventofcode.com/2023/day/19)

This is exactly the same as day 5, just in more dimensions and with a worse input format. The image through a piecewise linear map of a finite union of compacts is still a finite union of compacts, even two weeks later.

In [21]:
@dataclass
class Region:
    coord: dict[str, tuple[int, int]]
    MIN, MAX = 1, 4000
  
    def split_along(self, axis, val):
        clo, chi = self.coord[axis]
        return tuple(
            Region(self.coord | {axis: (nlo, nhi)}) if nlo <= nhi else None
            for nlo, nhi in ((clo, min(chi, val-1)), (max(clo, val), chi))
        )
    
    @classmethod
    def from_text(cls, s):
        return cls({k: (int(v), int(v)) for k, v in (u.split('=') for u in s.strip('{}').split(','))})
    
    whole = lambda c: Region({x: (Region.MIN, Region.MAX) for x in c})
      
    
@dataclass
class Workflow:
    name: str
    rules: list[tuple]
    BODY = re.compile(r'(\w+){.*,(\w+)}')
    GROUPS = re.compile(r'(\w+)([<>])(\d+):(\w+)')
    
    @classmethod
    def from_text(cls, s):
        name, dest = cls.BODY.match(s).groups()
        rules = cls.GROUPS.findall(s)
        return cls(
            name,
            [(var, op, int(val), dest) for var, op, val, dest in rules] + 
            [(None, None, None, dest)]
        )
                
    def apply_regions(self, regions):
        out = defaultdict(list)
        for rule in self.rules:
            new_regions = []
            for region in regions:
                match rule:
                    case None, None, None, dest:
                        out[dest].append(region)
                    case var, '>', val, dest:
                        lo, hi = region.split_along(var, val+1)
                        if lo: new_regions.append(lo)
                        if hi: out[dest].append(hi)
                    case var, '<', val, dest:
                        lo, hi = region.split_along(var, val)
                        if lo: out[dest].append(lo)
                        if hi: new_regions.append(hi)
            regions = new_regions
        return out


def apply_workflows_regions(ws, regions_start):
    while regions_start:
        loc, regions = regions_start.popitem()
        for d, r in ws[loc].apply_regions(regions).items():
            if d == 'A': yield from r
            elif d != 'R': regions_start[d] += r


workflows, parts = aocin(19).split('\n\n')
parts = list(map(Region.from_text, parts.split('\n')))
workflows = {w.name: w for w in map(Workflow.from_text, workflows.split('\n'))}

A = sum(
    a
    for k in (apply_workflows_regions(workflows, defaultdict(list, {'in': parts})))
    for a, _ in k.coord.values()
)
assert A == 373302

A = sum(
    prod(hi-lo+1 for lo, hi in u.coord.values())
    for u in apply_workflows_regions(workflows, defaultdict(list, {'in': [Region.whole('xmas')]}))
)
assert A == 130262715574114